## Loading Data ..

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset='/content/drive/MyDrive/Segmented_Augmented'
print(dataset)

In [ ]:
cd '/content/drive/MyDrive'

In [ ]:
pip install -U tensorflow-addons

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
##

from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
from random import shuffle, choice
import os
from PIL import Image
import matplotlib as plt
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import random
import glob
import time
import pandas as pd
import math
from random import randint
import numpy as np
import argparse
import os
import sys
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import matplotlib 
##

In [ ]:
def load_rgb_data(IMAGE_DIRECTORY,IMAGE_SIZE, shuffle=True):
    print("Loading images...")
    data = [] #empty list
    #categories = []
    directories = next(os.walk(IMAGE_DIRECTORY))[1]
   
    print(directories)
    j=0
    for diretcory_name in directories:

        print("Loading {0}".format(diretcory_name))
        file_names = next(os.walk(os.path.join(IMAGE_DIRECTORY, diretcory_name)))[2]
        print("we will load [", len(file_names), "] files from [",diretcory_name,"] class ..." )

        for i in range(len(file_names)):

          image_name = file_names[i]
          image_path = os.path.join(IMAGE_DIRECTORY, diretcory_name, image_name)
          if ('.DS_Store' not in image_path):
            
            category = diretcory_name
            label = j
            
            #get the image
            img = Image.open(image_path)
            #Images are in RGB format (PIL library)
            rgbimg = Image.new("RGB", img.size)
            rgbimg.paste(img) 
            img=rgbimg
            #resize the image to be the desired input size of the NN
            img = img.resize((IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)
            #convert images into np array (input of a neural network)
            data.append([np.array(img), label])
            #data.append([np.array(img), category])
        j=j+1

    if (shuffle):
      random.shuffle(data)
    images = np.array([i[0] for i in data]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
    labels = np.array([i[1] for i in data])
    
    print("File loading completed.")

    return images, labels, data

X_Data, Y_Data , Data = load_rgb_data(dataset, 224 , shuffle=True)
 
#X_Data = X_Data /255.0

print(X_Data.shape)
print(Y_Data.shape)

**Train, Test, Validation sets..**

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import cv2
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
# set some paths
model_dir = Path('/content/drive/MyDrive')

In [ ]:
'''
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
'''

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_Data, Y_Data, test_size=0.1,  stratify=Y_Data )
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train) # 0.25 x 0.8 = 0.2
X_Data =[]
Y_Data =[]
#one hit encoding
#y_train = np_utils.to_categorical(y_train)
#y_test = np_utils.to_categorical(y_test)
#y_val = np_utils.to_categorical(y_val)

## HCT

In [ ]:
positional_emb = True
conv_layers = 2
projection_dim = 512

num_heads = 1
transformer_units = [
    projection_dim,
    projection_dim,
]
transformer_layers = 1
stochastic_depth_rate = 0.001

learning_rate =0.000001
weight_decay = 0.0001
batch_size = 32
num_epochs = 200
image_size =  224

In [ ]:
class CCTTokenizer(layers.Layer):
  
  def __init__(
        self,
        kernel_size=3,
        stride=1,
        padding=1,
        pooling_kernel_size=3,
        pooling_stride=2,
        num_conv_layers=conv_layers,
        num_output_channels=[64, 128],
        positional_emb=positional_emb,
        **kwargs,
    ):
       super(CCTTokenizer, self).__init__(**kwargs)
       self.positional_emb = positional_emb
       
  def call(self, images):
      outputs = backbone(images)
        # After passing the images through our mini-network the spatial dimensions
        # are flattened to form sequences.
      reshaped = tf.reshape(outputs,
                            (-1, tf.shape(outputs)[1] * tf.shape(outputs)[2], tf.shape(outputs)[-1]),)
      return reshaped

  def positional_embedding(self, image_size):
        # Positional embeddings are optional in CCT. Here, we calculate 
        # the number of sequences and initialize an `Embedding` layer to
        # compute the positional embeddings later.
    if self.positional_emb:
      dummy_inputs = tf.ones((1, image_size, image_size, 3))
      dummy_outputs = self.call(dummy_inputs)
      sequence_length = tf.shape(dummy_outputs)[1]
      projection_dim = tf.shape(dummy_outputs)[-1]
      embed_layer = layers.Embedding(
          input_dim=sequence_length, output_dim=projection_dim
          )
      return embed_layer, sequence_length
    else:
      return None

In [ ]:
# Referred from: github.com:rwightman/pytorch-image-models.
class StochasticDepth(layers.Layer):
    def __init__(self, drop_prop, **kwargs):
        super(StochasticDepth, self).__init__(**kwargs)
        self.drop_prob = drop_prop

    def call(self, x, training=None):
        if training:
            keep_prob = 1 - self.drop_prob
            shape = (tf.shape(x)[0],) + (1,) * (len(tf.shape(x)) - 1)
            random_tensor = keep_prob + tf.random.uniform(shape, 0, 1)
            random_tensor = tf.floor(random_tensor)
            return (x / keep_prob) * random_tensor
        return x

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.relu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
# Note the rescaling layer. These layers have pre-defined inference behavior.

data_augmentation = keras.Sequential(
    [
        layers.Rescaling(scale=1.0 / 255),
        layers.RandomCrop(image_size, image_size),
        layers.RandomFlip("horizontal"),
    ],
    name="data_augmentation",
)


In [ ]:
backbone = tf.keras.applications.VGG19(weights="imagenet", input_shape=(224,224,3), include_top=False)
#backbone = tf.keras.applications.ResNet50(weights="imagenet", input_shape=(224,224,3), include_top=False)
#backbone = tf.keras.applications.EfficientNetB0(weights="imagenet", input_shape=(224,224,3), include_top=False)


In [ ]:
for layer in backbone.layers:
    layer.trainable = False

In [ ]:
num_classes = 3
input_shape = (224, 224, 3)
def create_cct_model(
    image_size=image_size,
    input_shape=input_shape,
    num_heads=num_heads,
    projection_dim=projection_dim,
    transformer_units=transformer_units,
    ):

    inputs = layers.Input(input_shape)

    # Augment data.
    augmented = data_augmentation(inputs)

    # Encode patches.
    cct_tokenizer = CCTTokenizer()
    encoded_patches = cct_tokenizer(inputs)

    # Apply positional embedding.
    if positional_emb:
        pos_embed, seq_length = cct_tokenizer.positional_embedding(image_size)
        positions = tf.range(start=0, limit=seq_length, delta=1)
        position_embeddings = pos_embed(positions)
        encoded_patches += position_embeddings


    # Calculate Stochastic Depth probabilities.
    dpr = [x for x in np.linspace(0, stochastic_depth_rate, transformer_layers)]

    # Create multiple layers of the Transformer block.
    for i in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-5)(encoded_patches)

        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)

        # Skip connection 1.
        attention_output = StochasticDepth(dpr[i])(attention_output)
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-5)(x2)

        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)

        # Skip connection 2.
        x3 = StochasticDepth(dpr[i])(x3)
        encoded_patches = layers.Add()([x3, x2])

    # Apply sequence pooling.
    representation = layers.LayerNormalization(epsilon=1e-5)(encoded_patches)
    attention_weights = tf.nn.softmax(layers.Dense(1)(representation), axis=1)
    weighted_representation = tf.matmul(
        attention_weights, representation, transpose_a=True
    )
    weighted_representation = tf.squeeze(weighted_representation, -2)

    # Classify outputs.
    logits = tf.nn.softmax(layers.Dense(3)(weighted_representation))
    
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    
    #KFold
    weight_decay=learning_rate/num_epochs
    optimizer = tfa.optimizers.AdamW( learning_rate=learning_rate, weight_decay=weight_decay)
    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(
            from_logits=True),
            metrics=[
                     keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
                     keras.metrics.TopKCategoricalAccuracy(5, name="top-5-accuracy"),],)
  
    return model

In [ ]:

#KFold
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from tensorflow.keras.models import Sequential, save_model, load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, cohen_kappa_score, roc_auc_score

acc_per_fold = []
loss_per_fold = []
acc = [] 
sensitivity = []
specificity = []
precision = []
F1 = []
TPR = []
FPR = []
yhat_probs = []
yhat_classes= []
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1

for train, val in kfold.split(X_train, y_train):
#X_train[train] = X_train, X_train[val] = X_val, y_train[train] = y_train, y_train[val] = y_val
  model = create_cct_model()
  total = 0
  cm = 0
  checkpoint_callback = []
  checkpoint_filepath = ""
  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  def exp_decay(num_epochs):
      lrate = learning_rate * np.exp(-weight_decay)
      return lrate
  lr_rate = keras.callbacks.LearningRateScheduler(exp_decay)
  
  def checkpointF(fold_no):
    return {
        1: 'vision_____transformer.h5f',
        2: 'vision______transformer.h5f',
        3: 'vision_______transformer.h5f',
        4: 'vision________transformer.h5f'
        }.get(fold_no, 'vision_________transformer.h5f')
        

  checkpoint_filepath = checkpointF(fold_no)
  print(checkpoint_filepath)
  checkpoint_callback = keras.callbacks.ModelCheckpoint(
      model_dir.joinpath(checkpoint_filepath),
      monitor="val_accuracy",
      save_best_only=True,
      save_weights_only=True,
      )

#X_train[train] = X_train, X_train[val] = X_val, y_train[train] = y_train, y_train[val] = y_val

  history = model.fit(
      x=X_train[train],
      y=y_train[train],
      batch_size=batch_size,
      validation_data =(X_train[val], y_train[val]),
      validation_steps=len(X_train[val])//batch_size,
      epochs=num_epochs,
      callbacks = [checkpoint_callback, lr_rate]
      )
  
  model.load_weights(checkpoint_filepath)
  _, accuracy, top_5_accuracy = model.evaluate(X_test, y_test)
  print(f"Test accuracy: {round(accuracy * 100, 2)}%")
  print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")
  
  # Generate generalization metrics
  model.load_weights(checkpoint_filepath)
  scores = model.evaluate(X_test, y_test)
  #scores = model.evaluate(X_test, y_test, verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  yhat_probs.append(model.predict(X_test, verbose=0))
# predict crisp classes for test set
  yhat_classes.append(np.argmax(yhat_probs[fold_no-1], axis=1))
#yhat_probs = yhat_probs[:,0]

#Multiclass
  cm = confusion_matrix(y_test, yhat_classes[fold_no-1])
  total = sum(sum(cm))
  acc.append((cm[0,0] + cm[1, 1] + cm[2, 2]) / total)
  sensitivity.append(cm[2,2]/(cm[2,0] + cm[2,2] + cm[2,1]))
  specificity.append((cm[0,0]+ cm[0,1]+ cm[1,0]+ cm[1,1])/((cm[0,0]+ cm[0,1]+ cm[1,0] + cm[1,1] + cm[1,2]+cm[0,2]))
  precision.append(cm[2,2]/(cm[2,2] + cm[1,2]+cm[0,2]))
  F1.append(2*((precision[fold_no-1] * sensitivity[fold_no-1])/(precision[fold_no-1] + sensitivity[fold_no-1])))
  TPR.append(sensitivity[fold_no-1])
  FPR.append(1-specificity[fold_no-1])
  # Increase fold number
  fold_no = fold_no + 1

  # == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')


In [ ]:

acc_Avrg = 0
sensitivity_Avrg = 0
specificity_Avrg = 0
precision_Avrg = 0
F1_Avrg = 0
TPR_Avrg = 0
FPR_Avrg = 0
for i in range(5):
  acc_Avrg += acc[i]
  sensitivity_Avrg += sensitivity[i]
  specificity_Avrg += specificity[i]
  precision_Avrg += precision[i]
  F1_Avrg += F1[i]
  TPR_Avrg += TPR[i]
  FPR_Avrg += FPR[i]

acc_Avrg = acc_Avrg/5
print(f'acc_Avrg = {acc_Avrg}')
print(f'acc = {acc}')

sensitivity_Avrg = sensitivity_Avrg/5
print(f'sensitivity_Avrg = {sensitivity_Avrg}')
print(f'sensitivity = {sensitivity}')

specificity_Avrg = specificity_Avrg/5
print(f'specificity_Avrg = {specificity_Avrg}')
print(f'specificity = {specificity}')

precision_Avrg = precision_Avrg/5
print(f'precision_Avrg = {precision_Avrg}')
print(f'precision = {precision}')

F1_Avrg = F1_Avrg/5
print(f'F1_Avrg = {F1_Avrg}')
print(f'F1 = {F1}')

TPR_Avrg = TPR_Avrg/5
print(f'TPR_Avrg = {TPR_Avrg}')
print(f'TPR = {TPR}')

FPR_Avrg = FPR_Avrg/5
print(f'FPR_Avrg = {FPR_Avrg}')
print(f'FPR = {FPR}')


In [ ]:
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Train and Validation Losses Over Epochs", fontsize=14)
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.plot(history.history["accuracy"], label="train_accuracy")
plt.plot(history.history["val_accuracy"], label="val_accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Train and Validation Accuracy Over Epochs", fontsize=14)
plt.legend()
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, cohen_kappa_score, roc_auc_score
yhat_probs = cct_model.predict(X_test, verbose=0)
# predict crisp classes for test set
yhat_classes = np.argmax(yhat_probs, axis=1)
#yhat_probs = yhat_probs[:,0]

In [ ]:
# roc curve for classes
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

fpr = {}
tpr = {}
thresh ={}

n_class = 3

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_test, yhat_probs[:,i], pos_label=i)
    #roc_auc_score(y_test, yhat_probs, multi_class='ovo', average='weighted')
    #print(i)
    

# plotting    
plt.plot(fpr[0], tpr[0], linestyle='--',color='orange', label='Class CAP vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label='Class NORMAL vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='--',color='blue', label='Class CP vs Rest')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('Multiclass ROC',dpi=300);

In [ ]:
#Multiclass CM
cm = confusion_matrix(y_test, yhat_classes)
total = sum(sum(cm))
acc = (cm[0,0] + cm[1, 1] + cm[2, 2]) / total

sensitivity = cm[2,2]/(cm[2,0] + cm[2,2] + cm[2,1])
specificity = (cm[0,0]+ cm[0,1]+ cm[1,0]+ cm[1,1])/((cm[0,0]+ cm[0,1]+ cm[1,0] + cm[1,1] + cm[1,2]+cm[0,2])
precision = cm[2,2]/(cm[2,2] + cm[1,2]+cm[0,2])
F1 = 2*((precision * sensitivity)/(precision + sensitivity))
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))
print("precision: {:.4f}".format(precision))
print("f1: {:.4f}".format(F1))
cm_df = pd.DataFrame(cm,
                     index = ['CAP', 'NORMAL', 'CP'], 
                     columns = ['CAP', 'NORMAL', 'CP'])

In [ ]:
#Plotting the confusion matrix
import seaborn as sns;
plt.figure(figsize=(5,4))
sns.heatmap(cm_df, annot=True, cmap="Blues")
plt.title('Confusion Matrix')
plt.ylabel('Actal Values')
plt.xlabel('Predicted Values')
plt.show()

In [ ]:
#AUC value
from sklearn.metrics import roc_auc_score
r = roc_auc_score(y_test, yhat_probs, average='macro', sample_weight=None, max_fpr=None, multi_class='ovr', labels=None)
#fpr[i], tpr[i], thresh[i] = roc_curve(y_test, yhat_probs[:,i], pos_label=i)
print(r)

## Transfer Learning

In [ ]:
import tensorflow as tf
INIT_LR = .0001
epochs = 100
BatchSize = 64
backbone = tf.keras.applications.VGG19(weights="imagenet", input_shape=(224,224,3), include_top=False)
#backbone = tf.keras.applications.ResNet50(weights="imagenet", input_shape=(224,224,3), include_top=False)
#backbone = tf.keras.applications.EfficientNetB0(weights="imagenet", input_shape=(224,224,3), include_top=False)

In [ ]:
def create_model():
  headModel = baseModel.output
  headModel = AveragePooling2D(pool_size=(4,4))(headModel)
  headModel = Flatten(name="flatten")(headModel)
  headModel = Dense(64, activation="relu")(headModel)
  headModel = Dropout(0.5)(headModel)
  headModel = Dense(3, activation="softmax")(headModel)
  model = Model(inputs = baseModel.input, outputs=headModel)

  for layer in baseModel.layers:
    layer.trainable = False

  weight_decay=INIT_LR/ epochs
  optimizer = tfa.optimizers.AdamW( learning_rate=INIT_LR, weight_decay=weight_decay)
    
  model.compile(
      optimizer=optimizer,
      loss=keras.losses.SparseCategoricalCrossentropy(
          from_logits=False),
          metrics=[keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
                   keras.metrics.TopKCategoricalAccuracy(5, name="top-5-accuracy")])
  
  return model

In [ ]:
#KFold
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from tensorflow.keras.models import Sequential, save_model, load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, cohen_kappa_score, roc_auc_score

acc_per_fold = []
loss_per_fold = []
acc = [] 
sensitivity = []
specificity = []
precision = []
F1 = []
TPR = []
FPR = []

# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
weight_decay=INIT_LR/ epochs
for train, val in kfold.split(X_train, y_train):
#X_train[train] = X_train, X_train[val] = X_val, y_train[train] = y_train, y_train[val] = y_val
  model = create_model()
  total = 0
  cm = 0
  checkpoint_callback = []
  checkpoint_filepath = ""
  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  
  # Fit data to model
  def exp_decay(num_epochs):
    lrate = INIT_LR * np.exp(-weight_decay)
    return lrate

  lr_rate = keras.callbacks.LearningRateScheduler(exp_decay)
    
  
  def checkpointF(fold_no):
    return {
        1: 'covidCXRcnnN.h5f',
        2: 'covidCXRcnnNN.h5f',
        3: 'covidCXRcnnNNN.h5f',
        4: 'covidCXRcnnNNNN.h5f'
        }.get(fold_no, 'covidCXRcnnNNNNN.h5f')
        

  checkpoint_filepath = checkpointF(fold_no)
  print(checkpoint_filepath)
  checkpoint_callback = keras.callbacks.ModelCheckpoint(
      model_dir.joinpath(checkpoint_filepath),
      monitor="val_accuracy",
      save_best_only=True,
      save_weights_only=True,
      )

#X_train[train] = X_train, X_train[val] = X_val, y_train[train] = y_train, y_train[val] = y_val

  history = model.fit(
      x=X_train[train],
      y=y_train[train],
      batch_size=BatchSize,
      validation_data =(X_train[val], y_train[val]),
      validation_steps=len(X_train[val])//BatchSize,
      epochs=epochs,
      callbacks = [checkpoint_callback, lr_rate]
      )
  

  '''
  model.load_weights(checkpoint_filepath)
  _, accuracy, top_5_accuracy = model.evaluate(X_test, y_test)
  print(f"Test accuracy: {round(accuracy * 100, 2)}%")
  print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")
  '''
  # Generate generalization metrics
  model.load_weights(checkpoint_filepath)
  scores = model.evaluate(X_test, y_test)
  #scores = model.evaluate(X_test, y_test, verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  yhat_probs = model.predict(X_test, verbose=0)
# predict crisp classes for test set
  yhat_classes = np.argmax(yhat_probs, axis=1)
#yhat_probs = yhat_probs[:,0]

#Multiclass
  cm = confusion_matrix(y_test, yhat_classes)
  total = sum(sum(cm))
  acc.append((cm[0,0] + cm[1, 1] + cm[2, 2]) / total)
  sensitivity.append(cm[2,2]/(cm[2,0] + cm[2,2] + cm[2,1]))
  specificity.append((cm[0,0]+ cm[0,1]+ cm[1,0]+ cm[1,1])/((cm[0,0]+ cm[0,1]+ cm[1,0] + cm[1,1] + cm[1,2]+cm[0,2]))
  precision.append(cm[2,2]/(cm[2,2] + cm[1,2]+cm[0,2]))
  F1.append(2*((precision[fold_no-1] * sensitivity[fold_no-1])/(precision[fold_no-1] + sensitivity[fold_no-1])))
  TPR.append(sensitivity[fold_no-1])
  FPR.append(1-specificity[fold_no-1])
  # Increase fold number
  fold_no = fold_no + 1

  # == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [ ]:
acc_Avrg = 0
sensitivity_Avrg = 0
specificity_Avrg = 0
precision_Avrg = 0
F1_Avrg = 0
TPR_Avrg = 0
FPR_Avrg = 0
for i in range(5):
  acc_Avrg += acc[i]
  sensitivity_Avrg += sensitivity[i]
  specificity_Avrg += specificity[i]
  precision_Avrg += precision[i]
  F1_Avrg += F1[i]
  TPR_Avrg += TPR[i]
  FPR_Avrg += FPR[i]

acc_Avrg = acc_Avrg/5
print(f'acc_Avrg = {acc_Avrg}')
print(f'acc = {acc}')

sensitivity_Avrg = sensitivity_Avrg/5
print(f'sensitivity_Avrg = {sensitivity_Avrg}')
print(f'sensitivity = {sensitivity}')

specificity_Avrg = specificity_Avrg/5
print(f'specificity_Avrg = {specificity_Avrg}')
print(f'specificity = {specificity}')

precision_Avrg = precision_Avrg/5
print(f'precision_Avrg = {precision_Avrg}')
print(f'precision = {precision}')

F1_Avrg = F1_Avrg/5
print(f'F1_Avrg = {F1_Avrg}')
print(f'F1 = {F1}')

TPR_Avrg = TPR_Avrg/5
print(f'TPR_Avrg = {TPR_Avrg}')
print(f'TPR = {TPR}')

FPR_Avrg = FPR_Avrg/5
print(f'FPR_Avrg = {FPR_Avrg}')
print(f'FPR = {FPR}')

In [ ]:
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Train and Validation Losses Over Epochs", fontsize=14)
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.plot(history.history["accuracy"], label="train_accuracy")
plt.plot(history.history["val_accuracy"], label="val_accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Train and Validation Accuracy Over Epochs", fontsize=14)
plt.legend()
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, cohen_kappa_score, roc_auc_score
yhat_probs = cct_model.predict(X_test, verbose=0)
# predict crisp classes for test set
yhat_classes = np.argmax(yhat_probs, axis=1)
#yhat_probs = yhat_probs[:,0]

In [ ]:
# roc curve for classes
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

fpr = {}
tpr = {}
thresh ={}

n_class = 3

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_test, yhat_probs[:,i], pos_label=i)
    #roc_auc_score(y_test, yhat_probs, multi_class='ovo', average='weighted')
    #print(i)
    

# plotting    
plt.plot(fpr[0], tpr[0], linestyle='--',color='orange', label='Class CAP vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label='Class NORMAL vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='--',color='blue', label='Class CP vs Rest')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('Multiclass ROC',dpi=300);

In [ ]:
#Multiclass CM
cm = confusion_matrix(y_test, yhat_classes)
total = sum(sum(cm))
acc = (cm[0,0] + cm[1, 1] + cm[2, 2]) / total

sensitivity = cm[2,2]/(cm[2,0] + cm[2,2] + cm[2,1])
specificity = (cm[0,0]+ cm[0,1]+ cm[1,0]+ cm[1,1])/((cm[0,0]+ cm[0,1]+ cm[1,0] + cm[1,1] + cm[1,2]+cm[0,2])
precision = cm[2,2]/(cm[2,2] + cm[1,2]+cm[0,2])
F1 = 2*((precision * sensitivity)/(precision + sensitivity))
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))
print("precision: {:.4f}".format(precision))
print("f1: {:.4f}".format(F1))
cm_df = pd.DataFrame(cm,
                     index = ['CAP', 'NORMAL', 'CP'], 
                     columns = ['CAP', 'NORMAL', 'CP'])

In [ ]:
#Plotting the confusion matrix
import seaborn as sns;
plt.figure(figsize=(5,4))
sns.heatmap(cm_df, annot=True, cmap="Blues")
plt.title('Confusion Matrix')
plt.ylabel('Actal Values')
plt.xlabel('Predicted Values')
plt.show()

In [ ]:
#AUC value
from sklearn.metrics import roc_auc_score
r = roc_auc_score(y_test, yhat_probs, average='macro', sample_weight=None, max_fpr=None, multi_class='ovr', labels=None)
#fpr[i], tpr[i], thresh[i] = roc_curve(y_test, yhat_probs[:,i], pos_label=i)
print(r)

## CCT

In [ ]:
positional_emb = True
conv_layers = 2
projection_dim = 128

num_heads = 2
transformer_units = [
    projection_dim,
    projection_dim,
]
transformer_layers = 2
stochastic_depth_rate = 0.1

learning_rate = 0.001
weight_decay = 0.0001
batch_size = 128
num_epochs = 100
image_size = 32

In [ ]:
num_classes = 3
input_shape = (32, 32, 3)
class CCTTokenizer(layers.Layer):
    def __init__(
        self,
        kernel_size=3,
        stride=1,
        padding=1,
        pooling_kernel_size=3,
        pooling_stride=2,
        num_conv_layers=conv_layers,
        num_output_channels=[64, 128],
        positional_emb=positional_emb,
        **kwargs,
    ):
        super(CCTTokenizer, self).__init__(**kwargs)

        # This is our tokenizer.
        self.conv_model = keras.Sequential()
        for i in range(num_conv_layers):
            self.conv_model.add(
                layers.Conv2D(
                    num_output_channels[i],
                    kernel_size,
                    stride,
                    padding="valid",
                    use_bias=False,
                    activation="relu",
                    kernel_initializer="he_normal",
                )
            )
            self.conv_model.add(layers.ZeroPadding2D(padding))
            self.conv_model.add(
                layers.MaxPool2D(pooling_kernel_size, pooling_stride, "same")
            )

        self.positional_emb = positional_emb

    def call(self, images):
        outputs = self.conv_model(images)
        # After passing the images through our mini-network the spatial dimensions
        # are flattened to form sequences.
        reshaped = tf.reshape(
            outputs,
            (-1, tf.shape(outputs)[1] * tf.shape(outputs)[2], tf.shape(outputs)[-1]),
        )
        return reshaped

    def positional_embedding(self, image_size):
        # Positional embeddings are optional in CCT. Here, we calculate
        # the number of sequences and initialize an `Embedding` layer to
        # compute the positional embeddings later.
        if self.positional_emb:
            dummy_inputs = tf.ones((1, image_size, image_size, 3))
            dummy_outputs = self.call(dummy_inputs)
            sequence_length = tf.shape(dummy_outputs)[1]
            projection_dim = tf.shape(dummy_outputs)[-1]

            embed_layer = layers.Embedding(
                input_dim=sequence_length, output_dim=projection_dim
            )
            return embed_layer, sequence_length
        else:
            return None

In [ ]:
# Referred from: github.com:rwightman/pytorch-image-models.
class StochasticDepth(layers.Layer):
    def __init__(self, drop_prop, **kwargs):
        super(StochasticDepth, self).__init__(**kwargs)
        self.drop_prob = drop_prop

    def call(self, x, training=None):
        if training:
            keep_prob = 1 - self.drop_prob
            shape = (tf.shape(x)[0],) + (1,) * (len(tf.shape(x)) - 1)
            random_tensor = keep_prob + tf.random.uniform(shape, 0, 1)
            random_tensor = tf.floor(random_tensor)
            return (x / keep_prob) * random_tensor
        return x

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
# Note the rescaling layer. These layers have pre-defined inference behavior.
data_augmentation = keras.Sequential(
    [
        layers.Rescaling(scale=1.0 / 255),
        layers.RandomCrop(image_size, image_size),
        layers.RandomFlip("horizontal"),
    ],
    name="data_augmentation",
)

In [ ]:
num_classes = 3
input_shape = (32, 32, 3)
def create_cct_model(
    image_size=image_size,
    input_shape=input_shape,
    num_heads=num_heads,
    projection_dim=projection_dim,
    transformer_units=transformer_units,
    ):

    inputs = layers.Input(input_shape)

    # Augment data.
    augmented = data_augmentation(inputs)

    # Encode patches.
    cct_tokenizer = CCTTokenizer()
    encoded_patches = cct_tokenizer(inputs)

    # Apply positional embedding.
    if positional_emb:
        pos_embed, seq_length = cct_tokenizer.positional_embedding(image_size)
        positions = tf.range(start=0, limit=seq_length, delta=1)
        position_embeddings = pos_embed(positions)
        encoded_patches += position_embeddings


    # Calculate Stochastic Depth probabilities.
    dpr = [x for x in np.linspace(0, stochastic_depth_rate, transformer_layers)]

    # Create multiple layers of the Transformer block.
    for i in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-5)(encoded_patches)

        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)

        # Skip connection 1.
        attention_output = StochasticDepth(dpr[i])(attention_output)
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-5)(x2)

        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)

        # Skip connection 2.
        x3 = StochasticDepth(dpr[i])(x3)
        encoded_patches = layers.Add()([x3, x2])

    # Apply sequence pooling.
    representation = layers.LayerNormalization(epsilon=1e-5)(encoded_patches)
    attention_weights = tf.nn.softmax(layers.Dense(1)(representation), axis=1)
    weighted_representation = tf.matmul(
        attention_weights, representation, transpose_a=True
    )
    weighted_representation = tf.squeeze(weighted_representation, -2)

    # Classify outputs.
    logits = tf.nn.softmax(layers.Dense(3)(weighted_representation))
    
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    
    #222
    weight_decay=learning_rate/num_epochs
    optimizer = tfa.optimizers.AdamW( learning_rate=learning_rate, weight_decay=weight_decay)
    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(
            from_logits=True),
            metrics=[
                     keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
                     keras.metrics.TopKCategoricalAccuracy(5, name="top-5-accuracy"),],)


    return model

In [ ]:

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from tensorflow.keras.models import Sequential, save_model, load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, cohen_kappa_score, roc_auc_score

acc_per_fold = []
loss_per_fold = []
acc = [] 
sensitivity = []
specificity = []
precision = []
F1 = []
TPR = []
FPR = []
yhat_probs = []
yhat_classes= []
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1

for train, val in kfold.split(X_train, y_train):
#X_train[train] = X_train, X_train[val] = X_val, y_train[train] = y_train, y_train[val] = y_val
  model = create_cct_model()
  total = 0
  cm = 0
  checkpoint_callback = []
  checkpoint_filepath = ""
  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  def exp_decay(num_epochs):
      lrate = learning_rate * np.exp(-weight_decay)
      return lrate
  lr_rate = keras.callbacks.LearningRateScheduler(exp_decay)
  
  def checkpointF(fold_no):
    return {
        1: 'vision_____transformer.h5f',
        2: 'vision______transformer.h5f',
        3: 'vision_______transformer.h5f',
        4: 'vision________transformer.h5f'
        }.get(fold_no, 'vision_________transformer.h5f')
        

  checkpoint_filepath = checkpointF(fold_no)
  print(checkpoint_filepath)
  checkpoint_callback = keras.callbacks.ModelCheckpoint(
      model_dir.joinpath(checkpoint_filepath),
      monitor="val_accuracy",
      save_best_only=True,
      save_weights_only=True,
      )

#X_train[train] = X_train, X_train[val] = X_val, y_train[train] = y_train, y_train[val] = y_val

  history = model.fit(
      x=X_train[train],
      y=y_train[train],
      batch_size=batch_size,
      validation_data =(X_train[val], y_train[val]),
      validation_steps=len(X_train[val])//batch_size,
      epochs=num_epochs,
      callbacks = [checkpoint_callback, lr_rate]
      )
  
  # Generate generalization metrics
  model.load_weights(checkpoint_filepath)
  scores = model.evaluate(X_test, y_test)
  #scores = model.evaluate(X_test, y_test, verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  yhat_probs.append(model.predict(X_test, verbose=0))
# predict crisp classes for test set
  yhat_classes.append(np.argmax(yhat_probs[fold_no-1], axis=1))
#yhat_probs = yhat_probs[:,0]

#Multiclass
  cm = confusion_matrix(y_test, yhat_classes[fold_no-1])
  total = sum(sum(cm))
  acc.append((cm[0,0] + cm[1, 1] + cm[2, 2]) / total)
  sensitivity.append(cm[2,2]/(cm[2,0] + cm[2,2] + cm[2,1]))
  specificity.append((cm[0,0]+ cm[0,1]+ cm[1,0]+ cm[1,1])/((cm[0,0]+ cm[0,1]+ cm[1,0] + cm[1,1] + cm[1,2]+cm[0,2]))
  precision.append(cm[2,2]/(cm[2,2] + cm[1,2]+cm[0,2]))
  F1.append(2*((precision[fold_no-1] * sensitivity[fold_no-1])/(precision[fold_no-1] + sensitivity[fold_no-1])))
  TPR.append(sensitivity[fold_no-1])
  FPR.append(1-specificity[fold_no-1])
  # Increase fold number
  fold_no = fold_no + 1

  # == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')


In [ ]:
acc_Avrg = 0
sensitivity_Avrg = 0
specificity_Avrg = 0
precision_Avrg = 0
F1_Avrg = 0
TPR_Avrg = 0
FPR_Avrg = 0
for i in range(5):
  acc_Avrg += acc[i]
  sensitivity_Avrg += sensitivity[i]
  specificity_Avrg += specificity[i]
  precision_Avrg += precision[i]
  F1_Avrg += F1[i]
  TPR_Avrg += TPR[i]
  FPR_Avrg += FPR[i]

acc_Avrg = acc_Avrg/5
print(f'acc_Avrg = {acc_Avrg}')
print(f'acc = {acc}')

sensitivity_Avrg = sensitivity_Avrg/5
print(f'sensitivity_Avrg = {sensitivity_Avrg}')
print(f'sensitivity = {sensitivity}')

specificity_Avrg = specificity_Avrg/5
print(f'specificity_Avrg = {specificity_Avrg}')
print(f'specificity = {specificity}')

precision_Avrg = precision_Avrg/5
print(f'precision_Avrg = {precision_Avrg}')
print(f'precision = {precision}')

F1_Avrg = F1_Avrg/5
print(f'F1_Avrg = {F1_Avrg}')
print(f'F1 = {F1}')

TPR_Avrg = TPR_Avrg/5
print(f'TPR_Avrg = {TPR_Avrg}')
print(f'TPR = {TPR}')

FPR_Avrg = FPR_Avrg/5
print(f'FPR_Avrg = {FPR_Avrg}')
print(f'FPR = {FPR}')


In [ ]:
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Train and Validation Losses Over Epochs", fontsize=14)
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.plot(history.history["accuracy"], label="train_accuracy")
plt.plot(history.history["val_accuracy"], label="val_accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Train and Validation Accuracy Over Epochs", fontsize=14)
plt.legend()
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, cohen_kappa_score, roc_auc_score
yhat_probs = model.predict(X_test, verbose=0)
# predict crisp classes for test set
yhat_classes = np.argmax(yhat_probs, axis=1)

In [ ]:
# roc curve for classes
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

fpr = {}
tpr = {}
thresh ={}

n_class = 3

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_test, yhat_probs[:,i], pos_label=i)
    #roc_auc_score(y_test, yhat_probs, multi_class='ovo', average='weighted')
    #print(i)
    

# plotting    
plt.plot(fpr[0], tpr[0], linestyle='--',color='orange', label='Class CAP vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label='Class NORMAL vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='--',color='blue', label='Class CP vs Rest')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('Multiclass ROC',dpi=300);   

In [ ]:
#Multiclass CM
cm = confusion_matrix(y_test, yhat_classes)
total = sum(sum(cm))
acc = (cm[0,0] + cm[1, 1] + cm[2, 2]) / total

sensitivity = cm[2,2]/(cm[2,0] + cm[2,2] + cm[2,1])
specificity = (cm[0,0]+ cm[0,1]+ cm[1,0]+ cm[1,1])/((cm[0,0]+ cm[0,1]+ cm[1,0] + cm[1,1] + cm[1,2]+cm[0,2])
precision = cm[2,2]/(cm[2,2] + cm[1,2]+cm[0,2])
F1 = 2*((precision * sensitivity)/(precision + sensitivity))
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))
print("precision: {:.4f}".format(precision))
print("f1: {:.4f}".format(F1))
cm_df = pd.DataFrame(cm,
                     index = ['CAP', 'NORMAL', 'CP'], 
                     columns = ['CAP', 'NORMAL', 'CP'])

In [ ]:
#Plotting the confusion matrix
import seaborn as sns;
plt.figure(figsize=(5,4))
sns.heatmap(cm_df, annot=True, cmap="Blues")
plt.title('Confusion Matrix')
plt.ylabel('Actal Values')
plt.xlabel('Predicted Values')
plt.show()

In [ ]:
#AUC value
from sklearn.metrics import roc_auc_score
r = roc_auc_score(y_test, yhat_probs, average='macro', sample_weight=None, max_fpr=None, multi_class='ovr', labels=None)
#fpr[i], tpr[i], thresh[i] = roc_curve(y_test, yhat_probs[:,i], pos_label=i)
print(r)